<a href="https://colab.research.google.com/github/GithubPython7Eduard/Python-Library/blob/main/Memcoin_Graduation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Import the Libraries**

In [1]:
from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"edvardohanyan","key":"f14b36972ebf9dd31196c0c54d9e2c7b"}'}

In [2]:
import os
os.environ['KAGGLE_CONFIG_DIR'] = "/content"
!chmod 600 /content/kaggle.json

In [3]:
!kaggle competitions download -c solana-skill-sprint-memcoin-graduation

**Let's open the ZIP file in Colab**

In [4]:
import zipfile
with zipfile.ZipFile('/content/solana-skill-sprint-memcoin-graduation.zip', 'r') as zip_ref:
    zip_ref.extractall('/content')


**Import the Libraries**

In [44]:
import pandas as pd
import numpy as np
import lightgbm as lgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import log_loss

**Pre-processing of data**

In [27]:
# Download Datas
train = pd.read_csv('/content/train.csv')
test = pd.read_csv('/content/test_unlabeled.csv')
sample_submission = pd.read_csv('/content/sample_submission.csv')

In [28]:
# to see the first 5 rows
print(train.head())

   Unnamed: 0                                          mint   slot_min  \
0           1  BmTDA5HqcemLkEgpyK25sDhbvk652CTXjdWEa8fLpump  317876496   
1           3  4FJwryCAMMePNeWw9LTBXXfXABdkKcAxkTT6h9pdpump  317876500   
2           4  DeXchZLMzFm9nfvkfTKiXvBWWEpGMF6VDzvbMB6mpump  317876500   
3           5  6Mx4fnEPWpRLKtxY8VzVXo3CWwu2dvGp9BSJMWfVpump  317876500   
4           6  4rbGH5peYV3FvCfncfsfwtBNwpjiG36nZtvxoPZ9pump  317876500   

   slot_graduated  has_graduated  is_valid  
0             NaN          False      True  
1             NaN          False      True  
2             NaN          False      True  
3             NaN          False      True  
4             NaN          False      True  


In [29]:
# Data size (number of layers and columns)
print("Train Data Shape:", train.shape)
print("Test Data Shape:", test.shape)

Train Data Shape: (639557, 6)
Test Data Shape: (478832, 4)


In [30]:
# summary data
print(train.describe())

          Unnamed: 0      slot_min  slot_graduated
count  639557.000000  6.395570e+05    7.404000e+03
mean   443022.944580  3.193927e+08    3.194608e+08
std    230612.292544  8.661946e+05    8.723438e+05
min         1.000000  3.178765e+08    3.178766e+08
25%    249515.000000  3.186584e+08    3.186939e+08
50%    451408.000000  3.193919e+08    3.195730e+08
75%    642602.000000  3.201396e+08    3.201449e+08
max    827248.000000  3.209212e+08    3.237205e+08


In [31]:
# Let's check the column type
print(train.dtypes)


Unnamed: 0          int64
mint               object
slot_min            int64
slot_graduated    float64
has_graduated        bool
is_valid             bool
dtype: object


In [32]:
# Check for missing values
print(train.isnull().sum())


Unnamed: 0             0
mint                   0
slot_min               0
slot_graduated    632153
has_graduated          0
is_valid               0
dtype: int64


In [33]:
# Let's check the scale of missing values
missing_percentage = train['slot_graduated'].isnull().mean() * 100
print(f"Missing percentage for 'slot_graduated': {missing_percentage:.2f}%")

Missing percentage for 'slot_graduated': 98.84%


In [34]:
# We are deleting the 'slot_graded' line
train_data = train.drop(columns=['slot_graduated'])

train_data = train.dr

**Preparation for modeling**

In [35]:
# insert variables X and y
X = train_data.drop(columns=['mint', 'has_graduated'])
y = train_data['has_graduated']

In [40]:
# Removing special characters (for example, `: ` or other special characters) from the names of all variables
X.columns = X.columns.str.replace('[^A-Za-z0-9_]+', '', regex=True)

In [41]:
# Customizable training and validation groups
X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.2, random_state=42)

In [42]:
# Creating a LightGBM model
model = lgb.LGBMClassifier()

In [43]:
# Train the model
model.fit(X_train, y_train)

[LightGBM] [Info] Number of positive: 5932, number of negative: 505713
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.015411 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 510
[LightGBM] [Info] Number of data points in the train set: 511645, number of used features: 2
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.011594 -> initscore=-4.445608
[LightGBM] [Info] Start training from score -4.445608


LGBMClassifier()

Model Evaluation

In [45]:
# Predicting values
y_pred = model.predict_proba(X_valid)[:, 1]

# Log loss calculation
loss = log_loss(y_valid, y_pred)
print(f'Log Loss: {loss}')

Log Loss: 0.06280274675495337


**Presentation of results**

In [46]:
# Presenting the results
submission = pd.DataFrame({
    'mint': test['mint'],
    'has_graduated': model.predict_proba(test.drop(columns=['mint']))[:, 1]
})

In [50]:
# Save as CSV in the archive
submission.to_csv('submission.csv', index=False)

In [51]:
from google.colab import files
files.download('submission.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>